<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/main/openai_functioncalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
! pip install openai
! pip install python-dotenv
! pip install pydantic
! pip install langchain


! pip install google-generativeai

In [ ]:
import openai
from dotenv import load_dotenv
import os


In [ ]:
_= load_dotenv('drive/MyDrive/env')
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
class Tagging(BaseModel):
    """Extract substrings which have a format where alphanumeric characters are separated by forwar slashes.
    The limit on both sides should extend upto the point of encountering the first blank space.
    """
    accno: str = Field(description="extracted substring")


In [ ]:
convert_pydantic_to_openai_function(Tagging)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'Tagging',
 'description': 'Extract substrings which have a format where alphanumeric characters are separated by forwar slashes.\nThe limit on both sides should extend upto the point of encountering the first blank space.',
 'parameters': {'properties': {'accno': {'description': 'extracted substring',
    'type': 'string'}},
  'required': ['accno'],
  'type': 'object'}}

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract substring from input text as instructed"),
    ("user", "{input}")
])

In [ ]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [ ]:
tagging_chain = prompt | model_with_functions

In [ ]:
tagging_chain.invoke({"input": "My account number is CDAC/1/2345"})